In [19]:
# result = ts.get_hs300s()
# result.head()
# result.to_csv('/Users/zi-qinzhou/Documents/LearnBySelf/Quant/hs300.csv',encoding='utf-8-sig')
# data = pd.read_csv('hs300.csv') 
# print(result['name'].head())

In [2]:
import csv
import tushare as ts
import pandas as pd

from sklearn.ensemble import ExtraTreesClassifier

# Preprocessing Data
### 1. Downloading Data

In [3]:
data = ts.get_h_data('000300', index = True, start = '2015-01-01', end = '2016-01-01')

[Getting data:]####

In [4]:
data = data.sort_index(ascending=True)#升序
data.tail()

,open,high,close,low,volume,amount
date,,,,,,
2015-12-25,3832.085,3848.025,3838.201,3813.204,11962401300,163314511431
2015-12-28,3847.532,3853.389,3727.632,3727.632,15398844000,210026020459
2015-12-29,3723.050,3762.052,3761.875,3710.480,10188555500,140405054699
2015-12-30,3762.914,3765.657,3765.175,3726.278,10562995500,155744099319
2015-12-31,3760.904,3772.617,3731.005,3727.325,10187355600,145043696235


### 2. Features Fuction

In [22]:
def CCI(data, ndays):
    TP = (data['high'] + data['low'] + data['close']) / 3
    CCI = pd.Series((TP - pd.DataFrame.rolling(TP, ndays).mean()) / (0.015 * pd.DataFrame.rolling(TP, ndays).std()),
    name = 'CCI')
    result = pd.DataFrame(CCI)
    return result

def EVM(data, ndays):
    dm = ((data['high'] + data['low'])/2) - ((data['high'].shift(1) + data['low'].shift(1))/2)
    br = (data['volume'] / 100000000) / ((data['high'] - data['low']))
    EVM = dm / br
    EVM_MA = pd.Series(pd.DataFrame.rolling(EVM, ndays).mean(), name = 'EVM')
    result = pd.DataFrame(EVM_MA)
    return result

def SMA(data, ndays):
    SMA = pd.Series(pd.DataFrame.rolling(data['close'], ndays).mean(), name = 'SMA')
    result = pd.DataFrame(SMA)
    return result

def ROC(data,n):
    N = data['close'].diff(n)
    D = data['close'].shift(n)
    ROC = pd.Series(N/D,name='Rate of Change')
    result = pd.DataFrame(ROC)
    return result

def BBANDS(data, ndays):
    MA = pd.Series(pd.DataFrame.rolling(data['close'], ndays).mean())
    SD = pd.Series(pd.DataFrame.rolling(data['close'], ndays).std())

    b1 = MA + (2 * SD)
    B1 = pd.Series(b1, name = 'Upper BollingerBand')
    data = data.join(B1)

    b2 = MA - (2 * SD)
    B2 = pd.Series(b2, name = 'Lower BollingerBand')
    result = pd.DataFrame(B2)
    return result

def ForceIndex(data, ndays): 
    FI = pd.Series(data['close'].diff(ndays) * data['volume'], name = 'ForceIndex') 
    result = pd.DataFrame(FI) 
    return result

### 3. Building Training Datasets

In [23]:
final = pd.concat([CCI(data,5), EVM(data,5), SMA(data,5), ROC(data,5), BBANDS(data,5),ForceIndex(data,5)], axis=1)
final.dropna()

,CCI,EVM,SMA,Rate of Change,Lower BollingerBand,ForceIndex
date,,,,,,
2015-01-12,-105.969604,-8.018640,3580.8814,-0.035140,3463.650104,-3.411127e+12
2015-01-13,-62.966672,-8.473673,3555.4776,-0.034885,3448.880974,-2.377211e+12
2015-01-14,-57.391363,-9.535103,3527.2042,-0.038797,3478.402298,-2.770970e+12
2015-01-15,45.053627,-3.847704,3536.1766,0.012604,3453.304471,1.066874e+12
2015-01-16,108.078685,-0.651911,3553.8612,0.024931,3431.436611,2.499556e+12
2015-01-19,-98.533199,-15.008451,3522.1770,-0.045088,3303.477220,-5.220056e+12
2015-01-20,-70.835570,-18.308570,3498.6134,-0.033528,3251.931765,-3.509129e+12
2015-01-21,6.182364,-7.538331,3507.9058,0.013266,3257.041681,1.570082e+12
2015-01-22,38.098485,-8.268443,3500.6042,-0.010130,3261.912297,-9.815307e+11


In [27]:
temp_target = data.close.diff().apply(lambda x: 1 if x>=0 else 0)
temp_target

date
2015-01-05    0
2015-01-06    0
2015-01-07    1
2015-01-08    0
2015-01-09    0
2015-01-12    0
2015-01-13    1
2015-01-14    0
2015-01-15    1
2015-01-16    1
2015-01-19    0
2015-01-20    1
2015-01-21    1
2015-01-22    1
2015-01-23    1
2015-01-26    1
2015-01-27    0
2015-01-28    0
2015-01-29    0
2015-01-30    0
2015-02-02    0
2015-02-03    1
2015-02-04    0
2015-02-05    0
2015-02-06    0
2015-02-09    1
2015-02-10    1
2015-02-11    1
2015-02-12    1
2015-02-13    1
             ..
2015-11-20    0
2015-11-23    0
2015-11-24    1
2015-11-25    1
2015-11-26    0
2015-11-27    0
2015-11-30    1
2015-12-01    1
2015-12-02    1
2015-12-03    1
2015-12-04    0
2015-12-07    1
2015-12-08    0
2015-12-09    1
2015-12-10    0
2015-12-11    0
2015-12-14    1
2015-12-15    0
2015-12-16    0
2015-12-17    1
2015-12-18    1
2015-12-21    1
2015-12-22    1
2015-12-23    0
2015-12-24    0
2015-12-25    1
2015-12-28    0
2015-12-29    1
2015-12-30    1
2015-12-31    0
Name: close, dtype:

In [25]:
final['target'] = temp_target

In [28]:
final.dropna()

,CCI,EVM,SMA,Rate of Change,Lower BollingerBand,ForceIndex,target
date,,,,,,,
2015-01-12,-105.969604,-8.018640,3580.8814,-0.035140,3463.650104,-3.411127e+12,0
2015-01-13,-62.966672,-8.473673,3555.4776,-0.034885,3448.880974,-2.377211e+12,1
2015-01-14,-57.391363,-9.535103,3527.2042,-0.038797,3478.402298,-2.770970e+12,0
2015-01-15,45.053627,-3.847704,3536.1766,0.012604,3453.304471,1.066874e+12,1
2015-01-16,108.078685,-0.651911,3553.8612,0.024931,3431.436611,2.499556e+12,1
2015-01-19,-98.533199,-15.008451,3522.1770,-0.045088,3303.477220,-5.220056e+12,0
2015-01-20,-70.835570,-18.308570,3498.6134,-0.033528,3251.931765,-3.509129e+12,1
2015-01-21,6.182364,-7.538331,3507.9058,0.013266,3257.041681,1.570082e+12,1
2015-01-22,38.098485,-8.268443,3500.6042,-0.010130,3261.912297,-9.815307e+11,1
